## Production Allocation

The following model is for a specific instance of the production
allocation problem seen in the first lectures. We give here the primal
 with the instantiated numerical parameters.


$$
\begin{array}{*{6}{r}l}
\max & 5x_1&+&6x_2&+&8x_3&=z\\
&6x_1&+&5x_2&+&10x_3 &\leq 60\\
&8x_1&+&4x_2&+&4x_3&\leq 40\\
&4x_1&+&5x_2&+&6x_3&\leq 50\\
&x_1,x_2,x_3&\geq 0
\end{array}
$$
and its dual model
$$
\begin{array}{*{6}{r}l}
\min & 60y_1&+&40y_2&+&50y_3&=u\\
&6y_1&+&8y_2&+&4y_3 &\leq 5\\
&5y_1&+&4y_2&+&5y_3&\leq 6\\
&10y_1&+&4y_2&+&6y_3&\leq 8\\
&y_1,y_2,y_3&\geq 0
\end{array}
$$



### Analysis of the final tableau


Solving one of the two problems provides the solution also to the other
problem. The final tableau of the primal problem looks like this:

```text
|------+----+----+------+----+----+----+-----|
|   x1 | x2 | x3 |   s1 | s2 | s3 | -z |   b |
|------+----+----+------+----+----+----+-----|
|   ?  |  1 |  0 |  ?   |  0 |  ? |  0 |   7 |
|   ?  |  0 |  1 |  ?   |  0 |  ? |  0 | 5/2 |
|   ?  |  0 |  0 |  ?   |  1 |  ? |  0 |   2 |
|------+----+----+------+----+----+----+-----|
| -0.2 |  0 |  0 | -0.2 |  0 | -1 |  1 | -62 |
|------+----+----+------+----+----+----+-----|
```

The question marks are for the values that are not relevant for the goals
of this exercise.

We deduce that the primal solution is $x^*_1=0,x^*_2=7,x^*_3=2.5$ and the
dual solution is $y^*_1=0.2,y^*_2=0,y^*_3=1$. The objective value is
$z^*=u^*=62$.



The three numbers in the last row for the columns of the non basic
variables are called *reduced costs*. They indicate how much we
should make each product more expensive in order to be worth
manufacturing it.  The next three values are known as *shadow
  prices*. After a change of sign they give us the values of the dual
variables, which are interpreted as the *marginal value* of
increasing (or decreasing) the capacities of the resources (that is, the
value by which the objective function would improve if the constraint
were relaxed by one unit, which corresponds to buying one unit more of
resource). In our example, which seeks maximization, the marginal value
1 for the third slack variable corresponding to the third resource means
that the objective function would increase by 1 if we could have one
more unit of that resource.

It can be verified that in the
primal problem at the optimum the first and third resources are fully
exhausted, that is, their constraint is satisfied at the equality, while
there is *slack* for the second resource, that is, the constraint
holds with strict inequality.  Looking at the marginal values, we see
that the second resource has been given a zero valuation. This seems
plausible, since we are not using all the capacity that we have, we are
not willing to place much value on it (buying one more unit of that
resource would not translate in an improvement of the objective
function).

These results are captured by the Complementary Slackness theorem of
linear programming. If a constraint is not "binding" in the
optimal primal solution, the corresponding dual variable is zero in the
optimal solution to the dual model.  Similarly, if a constraint in the
dual model is not "binding" in the optimal solution to the dual
model, then the corresponding variable is zero in the optimal solution
to the primal model.

## Solving the model with Pyomo

Let's write the primal model in Python and solve it with Pyomo. Here is
the script:

In [42]:
#!/usr/bin/python3
import pyomo.environ as po

model = po.ConcreteModel("prod")

# declare decision variables
model.x1 = po.Var(domain=po.NonNegativeReals)
model.x2 = po.Var(domain=po.NonNegativeReals)
model.x3 = po.Var(domain=po.NonNegativeReals)

# declare objective
model.profit = po.Objective(
    expr = 5.0*model.x1 + 6.0*model.x2 + 8.0*model.x3,
    sense = po.maximize)

# declare constraints
model.demand = po.Constraint(expr = 6.0*model.x1 + 5.0*model.x2 + 10.0*model.x3 <= 60.0)
model.laborA = po.Constraint(expr = 8.0*model.x1 + 4.0*model.x2 + 4.0*model.x3 <= 40.0)
model.laborB = po.Constraint(expr = 4.0*model.x1 + 5.0*model.x2 + 6.0*model.x3 <= 50.0)

# solve
solver = po.SolverFactory('glpk')
results = solver.solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 62.0
  Upper bound: 62.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.019019126892089844
# ----------------------------------------------------------
#   Solution Information
# -------------------------------------

The documentation for the functions `po.Var()`,
`po.Objective()`, `po.Constraint()`, as well as
for all other functions in Pyomo is available from the
[Reference Manual](https://pyomo.readthedocs.io/en/stable/index.html) and more specifically from the
[Modeling Components API page](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/index.html).

If the code is in a file called `prod1_scip.py` then we can solve the
model by calling:

```bash
> python3 prod1_scip.py
```

Most of the information associated with a Pyomo model is
stored in a set of attributes. Some attributes are associated with the
variables of the model, some with the constraints of the model, and some
with the model itself. To access these attributes you have to use the
methods available under `ConcreteModel`.

In [51]:
if str(results.Solver.status) == 'ok':
    # display solution
    print("Profit = ", model.profit())
    print("Units of X1 = ", model.x1())
    print("Units of X2 = ", model.x2())
    print("Units of X3 = ", model.x3())
else:
    print("No Valid Solution Found")

TypeError: 'str' object is not callable

In [52]:
fstr = "   {0:7.2f} {1:7.2f} {2:7.2f} "

print("Constraint  value  lslack  uslack")
for c in [model.demand, model.laborA, model.laborB]:
    print(c, fstr.format(c(), c.lslack(), c.uslack()))

Constraint  value  lslack  uslack
demand      60.00     inf    0.00 
laborA      38.00     inf    2.00 
laborB      50.00     inf    0.00 


**Debugging models**

Before trusting a solution, it is important to verify that the model solved is the one we wanted. In order to assess this fact it is possible to look at the model that was created:

In [19]:
model.pprint()

3 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   7.0 :  None : False : False : NonNegativeReals
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.5 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5.0*x1 + 6.0*x2 + 8.0*x3

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body                      : Upper : Active
        None :  -Inf : 6.0*x1 + 5.0*x2 + 10.0*x3 :  60.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body                     : Upper : Active
    

*Exporting Model Data to a File*

Priting the instance of the problem to a file in a standard format is another way to assess the correctness of the instance generated when Pyomo puts together the parameters and the model. This process is very useful with implicit models as we will see later. Inspect
the files created by these lines:

In [38]:
model.write("prod.lp")
model.write("prod.mps")

('prod.mps', 4446399960)

The files `prod.lp` and `prod.mps` contain the problem in two different formats. 

Another form of debugging is to inspect more througly the output while solving

In [29]:
#po.SolverFactory('cbc').solve(model,tee=True)
#po.SolverFactory('glpk').solve(model,tee=True)

## Solver Parameters

It is possible to specify the parameters of the solver (for example to set a time limit) by specifying them before solving the model. For example, we can avoid preprocessing to occur and set the primal simplex as the solution method:

The list of parameters can be found in the documentation of each specific solver:

- [GLPK](https://en.wikibooks.org/wiki/GLPK/Using_GLPSOL) or [GLPK](http://www.maximalsoftware.com/solvopt/optglpk.html)
- [CBC](https://www.coin-or.org/Cbc/cbcuserguide.html) or type `cbc -?` from a command line
- [SCIP](https://scip.zib.de/doc/html/PARAMETERS.php)
- [Gurobi](https://www.gurobi.com/documentation/9.0/refman/parameters.html)
- [CPLEX](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.10.0/ilog.odms.cplex.help/CPLEX/homepages/refparameterscplex.html)


In [35]:
solver = po.SolverFactory('glpk')
solver.options['tmlim'] = 5 # seconds
results = solver.solve(model)

## Your Task

Try all these formats on the production allocation example above and check their contents. The MPS file is not very user friendly. This is
because the format is an old format when computer technology had much
more limitations than nowadays. 
The CPLEX-LP format is a more explicit version of the problem that may
be useful to check whether the model we implemented in Python is
actually the one we intended.


Try also different solvers and inspect their output. 
Perhaps the most complete screen output is the one produced by the solver SCIP.
You can find an explanation of it from a shell:

```bash
$ scip
SCIP> display display
```

If a letter appears in front of a display row, it indicates, which heuristic
found the new primal bound, a star representing an integral
LP-relaxation.  In addition, the output indicates the amount of
presolving that is conducted. SCIP finds the optimal solution in two iterations with the primal simplex method - we can enforce this with:

In [60]:
solver = po.SolverFactory('SCIP')
#model.setPresolve(SCIP_PARAMSETTING.OFF) # deactivate preprocessing
#model.setHeuristics(SCIP_PARAMSETTING.OFF) # deactivate heuristics
#model.disablePropagation() # deactivate preprocessing at nodes
#model.setCharParam("lp/initalgorithm", "p") 
#m.setParam("limits/gap", 1.0) # solving stops, if the relative gap = |primal - dual|/MIN(|dual|,|primal|) is below the given value
#m.setParam("limits/memory",32000) # maximal memory usage in MB; reported memory usage is lower than real memory usage! default: 8796093022208
#m.setParam("limits/time", 10) # maximal time in seconds to run
solver.options["lp/initalgorithm"] = "p"
results = solver.solve(model)

FileNotFoundError: [Errno 2] No such file or directory: '/var/folders/w9/sxcp2ljj4wq3fdhy78rf5djh0000gn/T/tmp4l1jtxf1.pyomo.sol'

## The Value of the Dual and Slack variables

The value of the dual and slack variables can be accessed by importing further information from the models. This has to be specified via [Suffixes](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html) and declared right after the instantiation of the `ConcreteModel`. Then the value of the dual variables is associated to the constraints of the model:

In [57]:
model.dual = po.Suffix(direction=po.Suffix.IMPORT)
solver = po.SolverFactory('glpk')
results = solver.solve(model)

    'pyomo.core.base.suffix.Suffix'>) on block prod with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().


In [59]:
fstr = "   {0:7.2f} {1:7.2f} {2:7.2f} {3:7.2f}"

print("Constraint  value  lslack  uslack    dual")
for c in [model.demand, model.laborA, model.laborB]:
    print(c, fstr.format(c(), c.lslack(), c.uslack(),model.dual[c]))

Constraint  value  lslack  uslack    dual
demand      60.00     inf    0.00    0.20
laborA      38.00     inf    2.00    0.00
laborB      50.00     inf    0.00    1.00
